In [3]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
import time

In [ ]:
'''
1. 외부 서버 정의하기위해서 일단 기본적인 세팅
'''

'''
기준이되는 서버 적재되는 서버
'''

DB_HOST = "~"
DB_PORT = "5432"
DB_NAME = "airflow_db"
DB_USER = "airflow"
DB_PASSWORD = "airflow"

In [ ]:
'''
양쪽 서버에 fdw가 설치되어있어야함
'''

In [ ]:

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

table_query = """
SELECT * FROM pg_extension WHERE extname = 'postgres_fdw';

"""
query = table_query

try:
    postgres_fdw = pd.read_sql(query, engine)
    print("✅ Data retrieved successfully.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")
postgres_fdw   

In [94]:


engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

'''
밑에 정보는 cctv_db : 디토닉 데이터 마트 등록 정보 입력
'''

test_q = '''
CREATE SERVER remote_postgre
FOREIGN DATA WRAPPER postgres_fdw
OPTIONS (host '10.254.30.22', port '5432', dbname 'airflow');
'''

try:
    with engine.connect() as conn:
        result = conn.execute(text(test_q))  # ✅ text()로 감싸기
    print("✅ Query 실행 성공!")
except Exception as e:
    print(f"❌ Query 에러 발생: {e}")

✅ Query 실행 성공!


# 제거하는 부분

In [93]:
'''
제거하고 다시 테스트하는 부분
'''

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

'''
밑에 정보는 cctv_db : 디토닉 데이터 마트 등록 정보 입력
'''

test_q = '''
DROP SERVER remote_postgre CASCADE;
'''

try:
    with engine.connect() as conn:
        result = conn.execute(text(test_q))  # ✅ text()로 감싸기
    print("✅ Query 실행 성공!")
except Exception as e:
    print(f"❌ Query 에러 발생: {e}")

✅ Query 실행 성공!


# 등록 확인

In [95]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

query_d = f"""
SELECT * FROM pg_foreign_server;
"""

try:
    chk_server = pd.read_sql(query_d, engine)
    print("✅ Data retrieved successfully.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")
chk_server   

✅ Data retrieved successfully.


,oid,srvname,srvowner,srvfdw,srvtype,srvversion,srvacl,srvoptions
0,17217,remote_postgre,10,16942,None,None,None,"[host=10.254.30.22, port=5432, dbname=airflow]"


# 사용자 등록

In [96]:
'''
2. 사용자 등록 : 데이터마트 정보 입력 (안되면 hub 정보입력)
'''

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

test_q = '''
CREATE USER MAPPING FOR airflow
SERVER remote_postgre
OPTIONS (user 'ntels', password 'ntels4175');
'''

try:
    with engine.connect() as conn:
        result = conn.execute(text(test_q))  # ✅ text()로 감싸기
    print("✅ Query 실행 성공!")
except Exception as e:
    print(f"❌ Query 에러 발생: {e}")

✅ Query 실행 성공!


In [97]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

query_d = f"""
select * from pg_user_mappings;
"""

try:
    chk_server1 = pd.read_sql(query_d, engine)
    print("✅ Data retrieved successfully.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")
chk_server1

✅ Data retrieved successfully.


,umid,srvid,srvname,umuser,usename,umoptions
0,17218,17217,remote_postgre,10,airflow,"[user=ntels, password=ntels4175]"


# 외부 테이블 생성

In [13]:
def generate_foreign_table_ddl(df, table_name, server_name, schema_name='public'):
    # PostgreSQL 타입 매핑
    type_map = {
        'double precision': 'float8',
        'integer': 'int4',
        'bigint': 'int8',
        'smallint': 'int2',
        'character varying': 'varchar',
        'character': 'char',
        'text': 'text',
        'boolean': 'bool',
        'date': 'date',
        'timestamp without time zone': 'timestamp',
        'timestamp with time zone': 'timestamptz',
        'numeric': 'numeric'
    }

    column_defs = []
    for _, row in df.iterrows():
        col_name = row['column_name']
        data_type = row['data_type']

        mapped_type = type_map.get(data_type.lower(), data_type.lower())

        # 가변길이 문자형 처리
        if mapped_type in ('varchar', 'char') and row['character_maximum_length']:
            mapped_type += f"({int(row['character_maximum_length'])})"

        column_defs.append(f'    {col_name} {mapped_type}')

    column_block = ',\n'.join(column_defs)

    ddl = f'''CREATE FOREIGN TABLE {schema_name}.{table_name}_fdw (
{column_block}
)
SERVER {server_name}
OPTIONS (schema_name '{schema_name}', table_name '{table_name}');'''
    
    return ddl


In [9]:
'''
데이터를 가져올 db 정보
'''

DB_HOST2 = "10.254.30.22"
DB_PORT2 = "5432"
DB_NAME2 = "airflow"
DB_USER2 = "ntels"
DB_PASSWORD2 = "ntels4175"

In [10]:
'''
유동인구 테이블명 확인 // 페스티벌 데이터로 확인 하는중
'''
table_schema = '스키마 넣기'
table_name = 'fest_data'


In [11]:
engine = create_engine(f"postgresql://{DB_USER2}:{DB_PASSWORD2}@{DB_HOST2}:{DB_PORT2}/{DB_NAME2}")

query_d = f"""
SELECT 
    column_name, 
    data_type, 
    character_maximum_length,
    numeric_precision,
    is_nullable
FROM information_schema.columns
WHERE table_schema = '{table_schema}' AND table_name = '{table_name}'
ORDER BY ordinal_position;
"""

try:
    chk_table = pd.read_sql(query_d, engine)
    print("✅ Data retrieved successfully.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")
chk_table   

✅ Data retrieved successfully.


,column_name,data_type,character_maximum_length,numeric_precision,is_nullable
0,id,character varying,None,None,NO
1,datasetid,character varying,None,None,YES
2,modified_at,timestamp with time zone,None,None,YES
3,charorg,character varying,None,None,YES
4,donorg,character varying,None,None,YES
5,festcontent,character varying,None,None,YES
6,festendtime,character varying,None,None,YES
7,festlocation,character varying,None,None,YES
8,festname,character varying,None,None,YES
9,feststarttime,character varying,None,None,YES


In [35]:
'''
해당 테이블명으로 데이터타입확인
'''
table_schema = '스키마'
table_name = '테이블명'


In [1]:
engine = create_engine(f"postgresql://{DB_USER2}:{DB_PASSWORD2}@{DB_HOST2}:{DB_PORT2}/{DB_NAME2}")

query_d = f"""
SELECT 
    column_name, 
    data_type, 
    character_maximum_length,
    numeric_precision,
    is_nullable
FROM information_schema.columns
WHERE table_schema = '{table_schema}' AND table_name = '{table_name}'
ORDER BY ordinal_position;
"""

try:
    chk_table2 = pd.read_sql(query_d, engine)
    print("✅ Data retrieved successfully.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")
chk_table2   

In [ ]:
'''
DDL 생성
'''

In [2]:
ddl = generate_foreign_table_ddl(
    df=chk_table,  # ← 앞서 조회한 DataFrame
    # table_name='gyeongnam_50cell',
    table_name='fest_data',
    server_name='remote_postgre',
    schema_name='data_service_broker'
)
print(ddl)

# 외부테이블 입력

NameError: name 'DB_USER' is not defined

In [90]:
'''
3. 외부테이블 생성
'''
'''
데이터 쌓을 DB
'''

DB_HOST = "~"
DB_PORT = "5432"
DB_NAME = "airflow_db"
DB_USER = "airflow"
DB_PASSWORD = "airflow"

test_q = '''
CREATE FOREIGN TABLE soss.fest_data_fdw (
    id varchar,
    datasetid varchar,
    modified_at timestamptz,
    charorg varchar,
    donorg varchar,
    festcontent varchar,
    festendtime varchar,
    festlocation varchar,
    festname varchar,
    feststarttime varchar,
    hostorg varchar,
    info varchar,
    lat varchar,
    lot varchar,
    numbaddress varchar,
    orgid varchar,
    orgname varchar,
    pagelink varchar,
    roadaddress varchar,
    tel varchar,
    wrttimeid varchar
)
SERVER remote_postgre
OPTIONS (schema_name 'data_service_broker', table_name 'fest_data');
'''

try:
    engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    with engine.connect() as conn:
        conn.execute(text(test_q))
    print("✅ 외부 테이블 등록 완료!")
except Exception as e:
    print(f"❌ Query 에러 발생: {e}")

✅ 외부 테이블 등록 완료!


In [91]:

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

table_query = """
SELECT foreign_table_schema, foreign_table_name
FROM information_schema.foreign_tables;
"""
query = table_query

try:
    dblink = pd.read_sql(query, engine)
    print("✅ Data retrieved successfully.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")
dblink   

✅ Data retrieved successfully.


,foreign_table_schema,foreign_table_name
0,soss,fest_data_fdw


In [98]:

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

table_query = """
SELECT * from soss.fest_data_fdw
"""
query = table_query

try:
    t2 = pd.read_sql(query, engine)
    print("✅ Data retrieved successfully.")
except Exception as e:
    print(f"❌ Error fetching data: {e}")
t2

✅ Data retrieved successfully.


,id,datasetid,modified_at,charorg,donorg,festcontent,festendtime,festlocation,festname,feststarttime,...,info,lat,lot,numbaddress,orgid,orgname,pagelink,roadaddress,tel,wrttimeid
0,urn:datahub:fest_data:금산인삼축제:2024-11-25:B551011,fest_data,2025-03-26 06:11:01.596421+00:00,(재)금산축제관광재단,None,건강체험관+인삼캐기체험여행+인삼저잣거리+인삼아트체험+거리의 라디오쇼+금산인삼왕선발대회 등,2024-10-13,금삼세계인삼엑스포 광장+인삼약초거리 일원,금산인삼축제,2024-10-03,...,None,36.100014,127.50073,충청남도 금산군 금산읍 신대리 392,B551011,한국관광공사,http://www.insamfestival.co.kr/html/kr/index.html,충청남도 금산군 금산읍 인삼광장로 30,041-750-2307,2024-11-25
1,urn:datahub:fest_data:제24회 흥부골 달맞이 행사:2024-12-...,fest_data,2025-03-26 06:11:01.596421+00:00,인월자율방범대,전북특별자치도 남원시+전북특별자치도립국악원,터울림 및 제례+국악공연+달집태우기+소원소지,2025-02-12,남원시 인월면 람천둔치,제24회 흥부골 달맞이 행사,2025-02-12,...,None,35.418053,127.626625,전북특별자치도 남원시 산내면 대정리,4701000,전북특별자치도 남원시,None,None,063-620-4061,2024-12-13
2,urn:datahub:fest_data:제17회 DMZ국제다큐멘터리 영화제:2025...,fest_data,2025-03-26 06:11:01.596421+00:00,사단법인 DMZ국제다큐멘터리영화제,경기도+경기도 파주시청+문화체육관광부+영화진흥위원회,다큐멘터리 상영 및 시상+포럼+다큐강연 등,2025-09-24,임진각 평화누리 야외공연장,제17회 DMZ국제다큐멘터리 영화제,2025-09-18,...,None,37.889538,126.74019,경기도 파주시 문산읍 사목리 494-1,4060000,경기도 파주시,https://dmzdocs.com/,경기도 파주시 문산읍 임진각로 177,031-940-4723,2025-01-14
3,urn:datahub:fest_data:남부면 수국축제:2024-11-11:5370000,fest_data,2025-03-26 06:11:01.596421+00:00,남부면발전협의회,None,식전행사+화합행사+수국포토존+목재만들기 체험 등,2024-06-30,매물도 여객선터미널 및 남부면 저구항 일원,남부면 수국축제,2024-06-01,...,None,34.73225,128.61012,경상남도 거제시 남부면 저구리 70,5370000,경상남도 거제시,None,경상남도 거제시 남부면 남부해안로 30,055-639-6532,2024-11-11
4,urn:datahub:fest_data:의령 신번문화축제:2024-11-28:539...,fest_data,2025-03-26 06:11:01.596421+00:00,경상남도 의령군,경상남도 의령군 의령군의회,천년을 이어온 한지의 본고장이자 악성 우륵과 가야금의 탄생지인 부림면(옛 성열현)에...,2024-09-08,우륵문화마당,의령 신번문화축제,2024-09-06,...,None,35.322372,128.2614,경상남도 의령군 부림면 신반리 398,5390000,경상남도 의령군,https://www.uiryeong.go.kr/tour/board/view.uir...,경상남도 의령군 의령읍 충익로63,055-570-2505,2024-11-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,urn:datahub:fest_data:제9회 긴고랑 한마음 축제:2024-11-0...,fest_data,2025-03-26 06:11:01.596421+00:00,중곡4동 문화행사위원회,서울특별시 광진구,자치회관 프로그램 발표+ 주민 노래자랑+ 먹거리장터+ 체험 부스 운영 등,2023-10-05,긴고랑체육공원,제9회 긴고랑 한마음 축제,2023-10-05,...,None,None,None,서울특별시 광진구 중곡동 68-19,3040000,서울특별시 광진구,None,서울특별시 광진구 긴고랑로 143 긴고랑어린이공원,02-450-1072,2024-11-07
1256,urn:datahub:fest_data:제17회 차차동인전:2024-11-30:43...,fest_data,2025-03-26 06:11:01.596421+00:00,충청북도 충주시청,None,차차동인회 회원전,2024-11-18,충주시문화회관,제17회 차차동인전,2024-11-12,...,None,36.97062,127.937225,충청북도 충주시 성내동 215,4390000,충청북도 충주시,https://www.chungju.go.kr/culture/,충청북도 충주시 중앙로 128,043-850-5962,2024-11-30
1257,urn:datahub:fest_data:관악구 협치과제 <S1472 어린이주간>:2...,fest_data,2025-03-26 06:11:01.596421+00:00,서울특별시 관악구청,None,가정의 달 5월+ 어린이와 가족 관람객들이 ‘별빛내린천’과 ‘관천로 문화플랫폼 S1...,2023-05-07,관천로 문화플랫폼 S1472 및 봉림교 주변,관악구 협치과제 <S1472 어린이주간>,2023-05-05,...,None,37.48475,126.92508,서울특별시 관악구 신림동 1472-1,3200000,서울특별시 관악구,https://gfac.or.kr/,서울특별시 관악구 관천로 53,02-6357-5001,2024-10-15
1258,urn:datahub:fest_data:제9회 사천비토섬별주부전축제:2024-11-...,fest_data,2025-03-26 06:11:01.596421+00:00,별주부전축제추진위원회,None,용왕제+공연행사+ 체험행사+ 사생대회+ 먹거리 장터 등,2024-04-13,서포면 일원,제9회 사천비토섬별주부전축제,2024-04-12,...,None,None,None,경상남도 사천시 서포면 비토리 14-13,5340000,경상남도 사천시,None,경상남도 사천시 거북길 455,None,2024-11-15


# fdw 정의후 sql로 작업 하는 명령어

In [ ]:
'''
테이블명, ip, port, id,pw 확인 후 진행 
'''

conn = psycopg2.connect(
    host="~",
    port="~",
    dbname="~",
    user="~",
    password="~"
)
cur = conn.cursor()

query = f"""
INSERT INTO 스키마명.test ("grid_id", "yyyymmdd", "hh", "tt")
SELECT id AS grid_id, etl_ymd AS yyyymmdd, timezn_cd AS hh,
       (t10 + t15) AS tt
FROM 스키마.테이블명_fdw;
"""

cur.execute(query)
conn.commit()